In [1]:
import cv2
import numpy as np

In [2]:
# load the COCO class names
with open('./src/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')
    
# get a different color array for each of the classes
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))
print(class_names)

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', '']


In [3]:
# load the DNN model
model = cv2.dnn.readNet(model='./src/frozen_inference_graph.pb',
                        config='./src/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt',framework='TensorFlow')

In [4]:
# read the image from disk
image = cv2.imread('./img/cows.jpg')
image_height, image_width, _ = image.shape
# create blob from image
blob = cv2.dnn.blobFromImage(image=image, swapRB=True, crop=False)
# set the blob to the model
model.setInput(blob)
# forward pass through the model to carry out the detection
output = model.forward(["detection_out_final", "detection_masks"])
(boxes, masks) = output



In [5]:
print (len(masks))

100


In [6]:
def draw_boxes_masks(image, boxes, masks, labels, confidence=0.5, threshold=0.3):
    clone = image.copy()
    rois = []
    for i in range(0, boxes.shape[2]):
        classID = int(boxes[0, 0, i, 1])
        score = boxes[0, 0, i, 2]
        if score > confidence:
            (H, W) = clone.shape[:2]
            box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])

            (startX, startY, endX, endY) = box.astype("int")
            boxW = endX - startX
            boxH = endY - startY

            mask = masks[i, classID]
            mask = cv2.resize(mask, (boxW, boxH),interpolation=cv2.INTER_CUBIC)
            mask = (mask > threshold)

            roi = clone[startY:endY, startX:endX]

            roi = roi[mask]

            color = COLORS[classID]

            blended = ((0.4 * color) + (0.6 * roi)).astype("uint8")

            clone[startY:endY, startX:endX][mask] = blended

            color = [int(c) for c in color]
            cv2.rectangle(clone, (startX, startY), (endX, endY), color, 2)

            text = "{}: {:.4f}".format(labels[classID], score)
            cv2.putText(clone, text, (startX, startY - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return clone

In [7]:
drawed_image = draw_boxes_masks(
    image,
    boxes,
    masks,
    class_names
)

In [8]:
cv2.imshow("Output", drawed_image)
cv2.waitKey(0)

-1

video

In [9]:
# import the necessary packages

import cv2
import numpy as np
import time

# load the COCO class names
with open('./src/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')
    
# get a different color array for each of the classes
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

# load the DNN model
model = cv2.dnn.readNet(model='./src/frozen_inference_graph.pb',
                        config='./src/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt',framework='TensorFlow')



vs = cv2.VideoCapture('./video/vtest.avi')

while True:
    (grabbed, frame) = vs.read()
    if not grabbed:
        break

    start = time.time()
    blob = cv2.dnn.blobFromImage(image=frame, swapRB=True, crop=False)
    # set the blob to the model
    model.setInput(blob)
    # forward pass through the model to carry out the detection
    output = model.forward(["detection_out_final", "detection_masks"])
    (boxes, masks) = output
    end = time.time()

    drawed_image = draw_boxes_masks(
        frame,
        boxes,
        masks,
        class_names
    )

    cv2.imshow('image', drawed_image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break 
vs.release()
cv2.destroyAllWindows()